# Credit Risk Modelling
Credit risk refers to the risk of loss faced by financial institutions or lenders due to the possibility that a borrower may fail to repay their debt. It is one of the major risks in lending or investing, as lenders or investors may be exposed to the risk of not recovering the principal or receiving the interest payments.

I will be conducting credit risk modeling using the Lending Club dataset. Lending Club is a US-based peer-to-peer (P2P) lending company that matches investors with borrowers. Investors invest their funds through Lending Club, and these funds are then loaned to borrowers. When the borrower repays the loan, the principal and interest are returned to the investor. This creates a mutually beneficial scenario, where borrowers typically receive lower interest rates, and investors can achieve higher returns.

The dataset includes all available data for over 800,000 consumer loans issued by Lending Club between 2007 and 2015. It contains the current loan status ("loan_status") and the most recent payment information. Key features (variables) in the dataset include the number of financial inquiries ("inq_fi"), address information ("addr_state", "zip_code", etc.), and collection status ("collections"). The collections variable indicates whether the borrower has missed one or more payments and whether the team is actively trying to recover the funds.

In this project, we will calculate the Expected Loss (EL) to analyze credit risk. Expected Loss is derived by multiplying the following three key components: Probability of Default (PD), Loss Given Default (LGD), and Exposure at Default (EAD). Each of these factors will be modeled separately to provide a more accurate assessment of credit risk.

- PD (Probability of Default) model: This model will predict the probability of default, focusing on assessing the likelihood that a borrower will default on a loan.
- LGD (Loss Given Default) model: This model will estimate the actual loss rate in the event of a default, evaluating the extent of the loss when a loan defaults.
- EAD (Exposure at Default) model: This model will forecast the outstanding loan amount at the time of default, measuring the total balance remaining when a default occurs.

By modeling these components individually, we aim to provide a more detailed and precise evaluation of credit risk.

# Explore Data

In [1]:
# Import libraries
import pandas as pd
import numpy as np

In [2]:
# Load the data file
df_backup = pd.read_csv('loan_data_2007_2014.csv')

/Users/oliviachoi/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# Copy the data
df = df_backup.copy()

In [4]:
# Display first 5 records
df.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Display columns name
df.columns.values

array(['Unnamed: 0', 'id', 'member_id', 'loan_amnt', 'funded_amnt',
       'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade',
       'sub_grade', 'emp_title', 'emp_length', 'home_ownership',
       'annual_inc', 'verification_status', 'issue_d', 'loan_status',
       'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
       'recoveries', 'collection_recovery_fee', 'last_pymnt_d',
       'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint',
       'veri

In [6]:
# Drop unnecessary column
df = df.drop('Unnamed: 0', axis=1)

In [7]:
# Display data type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466285 entries, 0 to 466284
Data columns (total 74 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           466285 non-null  int64  
 1   member_id                    466285 non-null  int64  
 2   loan_amnt                    466285 non-null  int64  
 3   funded_amnt                  466285 non-null  int64  
 4   funded_amnt_inv              466285 non-null  float64
 5   term                         466285 non-null  object 
 6   int_rate                     466285 non-null  float64
 7   installment                  466285 non-null  float64
 8   grade                        466285 non-null  object 
 9   sub_grade                    466285 non-null  object 
 10  emp_title                    438697 non-null  object 
 11  emp_length                   445277 non-null  object 
 12  home_ownership               466285 non-null  object 
 13 

# Preprocessing

## Continous Variables

In [8]:
# Object data type
# Display unique values in the 'emp_length' column to understand its current format
df['emp_length'].unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', nan],
      dtype=object)

In [9]:
# Convert Object data to numeric
# Remove '+ years' from the 'emp_length' values
df['emp_length_int'] = df['emp_length'].str.replace('\+ years', '')

/var/folders/7l/hzhbdmks3snf1lxq26d5wcbh0000gn/T/ipykernel_4061/3796932139.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['emp_length_int'] = df['emp_length'].str.replace('\+ years', '')


In [10]:
# Display unique values after the first replacement to check intermediate results
df['emp_length_int'].unique()

array(['10', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', nan],
      dtype=object)

In [11]:
# Replace '< 1 year' with '0'
df['emp_length_int'] = df['emp_length_int'].str.replace('< 1 year', str(0))

# Replace 'n/a' with '0'
df['emp_length_int'] = df['emp_length_int'].str.replace('n/a', str(0))

# Remove ' years' from the values
df['emp_length_int'] = df['emp_length_int'].str.replace(' years', '')

# Remove ' year' from the values
df['emp_length_int'] = df['emp_length_int'].str.replace(' year', '')

In [12]:
# Check the data type of the second element in the 'emp_length_int' column
type(df['emp_length_int'][1])

str

In [13]:
# Display unique values in the 'emp_length_int' column to verify final format
df['emp_length_int'].unique()

array(['10', '0', '1', '3', '8', '9', '4', '5', '6', '2', '7', nan],
      dtype=object)

In [14]:
# Convert the 'emp_length_int' column to numeric type (integers)
df['emp_length_int'] = pd.to_numeric(df['emp_length_int'])

In [15]:
# Display unique values in the 'emp_length_int' column after conversion to numeric
df['emp_length_int'].unique()

array([10.,  0.,  1.,  3.,  8.,  9.,  4.,  5.,  6.,  2.,  7., nan])

In [16]:
# Do the same thing for 'term'
df['term'].unique()

array([' 36 months', ' 60 months'], dtype=object)

In [17]:
df['term_int'] = df['term'].str.replace(' months', '')
df['term_int'] = df['term_int'].str.replace(' ', '')

In [18]:
df['term_int'].unique()

array(['36', '60'], dtype=object)

In [19]:
df['term_int'] = pd.to_numeric(df['term_int'])

In [20]:
df['term_int'].unique()

array([36, 60])

In [21]:
df['term_int']

0         36
1         60
2         36
3         36
4         60
          ..
466280    60
466281    60
466282    60
466283    36
466284    36
Name: term_int, Length: 466285, dtype: int64

The earliest credit line date refers to the date when a borrower first opened a line of credit, such as a credit card or loan. In credit risk analysis, this date is significant because the longer someone has maintained a credit line without defaults, the better it reflects on their creditworthiness. For instance, if a borrower has an earliest credit line date from many years ago, they may be seen as more reliable due to their longer history of managing credit.

In [22]:
# Object data type
# These are string variables but they are actually supposed to be date variables
df['earliest_cr_line']

0         Jan-85
1         Apr-99
2         Nov-01
3         Feb-96
4         Jan-96
           ...  
466280    Apr-03
466281    Jun-97
466282    Dec-01
466283    Feb-03
466284    Feb-00
Name: earliest_cr_line, Length: 466285, dtype: object

In [23]:
# Convert 'earliest_cr_line' column from string format to datetime format using the pattern '%b-%y' (e.g., 'Jan-85'). 
df['earliest_cr_line_date'] = pd.to_datetime(df['earliest_cr_line'], format = '%b-%y')

In [24]:
# Check the data type of the second element in the 'earliest_cr_line_date' column
type(df['earliest_cr_line_date'][1])

pandas._libs.tslibs.timestamps.Timestamp

In [25]:
# We should set a reference date to calculate the time that has passed
# Subtract 'earliest_cr_line_date' from the reference date to calculate the duration since the earliest credit line
pd.to_datetime('2017-12-01') - df['earliest_cr_line_date']

0        12022 days
1         6819 days
2         5874 days
3         7974 days
4         8005 days
            ...    
466280    5358 days
466281    7488 days
466282    5844 days
466283    5417 days
466284    6513 days
Name: earliest_cr_line_date, Length: 466285, dtype: timedelta64[ns]

In [26]:
# Calculate the number of months since the earliest credit line date
df['mths_since_earliest_cr_line'] = round(pd.to_numeric((pd.to_datetime('2017-12-01') - df['earliest_cr_line_date']) / np.timedelta64(1, 'M')))

In [27]:
# Get a summary of statistics for the 'mths_since_earliest_cr_line' column
df['mths_since_earliest_cr_line'].describe()


count    466256.000000
mean        239.482430
std          93.974829
min        -612.000000
25%         183.000000
50%         225.000000
75%         285.000000
max         587.000000
Name: mths_since_earliest_cr_line, dtype: float64

- Count: the number of entries
- Mean: average number of months since the earliest credit line
- Standard Deviation: measure of the variability in the number of months
- Minimum: -612 months which indicates possible data issues or errors => need further investigation!
- 25th Percentile: 25% of the values are below this point
- 50th Percentile (Median): 50% of the values are below this point
- 75th Percentile: 75% of the values are below this point
- Maximum: the largest number of months since the earliest credit line

In [28]:
# Display rows where 'mths_since_earliest_cr_line' is negative
df.loc[df['mths_since_earliest_cr_line'] < 0, ['earliest_cr_line', 'earliest_cr_line_date', 'mths_since_earliest_cr_line']]

,earliest_cr_line,earliest_cr_line_date,mths_since_earliest_cr_line
1580,Sep-62,2062-09-01,-537.0
1770,Sep-68,2068-09-01,-609.0
2799,Sep-64,2064-09-01,-561.0
3282,Sep-67,2067-09-01,-597.0
3359,Feb-65,2065-02-01,-566.0
...,...,...,...
464003,Jan-68,2068-01-01,-601.0
464260,Jul-66,2066-07-01,-583.0
465100,Oct-67,2067-10-01,-598.0
465500,Sep-67,2067-09-01,-597.0


When I converted the 'earliest_cr_line' column from string format to datetime format, the conversion did not work as expected. For example, 'Sep-62' was converted to 2062-09-01, whereas it should have been 1962-09-01. This issue aroce in the first place because the origin of the built-in time scale starts after 1970.

To resolve this, let's substitute the negative values with the maximum observed difference.

In [29]:
# Replace negative values in 'mths_since_earliest_cr_line' with the maximum value of the column
df['mths_since_earliest_cr_line'][df['mths_since_earliest_cr_line'] < 0] = df['mths_since_earliest_cr_line'].max()

/var/folders/7l/hzhbdmks3snf1lxq26d5wcbh0000gn/T/ipykernel_4061/342213293.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mths_since_earliest_cr_line'][df['mths_since_earliest_cr_line'] < 0] = df['mths_since_earliest_cr_line'].max()


In [30]:
# Check if there are any remaining negative values in the 'mths_since_earliest_cr_line' column after imputation
min(df['mths_since_earliest_cr_line'])

73.0

In [31]:
# Do the same thing for issue_d

In [32]:
# Object data type
df['issue_d']

0         Dec-11
1         Dec-11
2         Dec-11
3         Dec-11
4         Dec-11
           ...  
466280    Jan-14
466281    Jan-14
466282    Jan-14
466283    Jan-14
466284    Jan-14
Name: issue_d, Length: 466285, dtype: object

In [33]:
# Convert 'issue_d' column from string format to datetime format using the pattern '%b-%y' (e.g., 'Jan-85'). 
df['issue_d_date'] = pd.to_datetime(df['issue_d'], format = '%b-%y')

In [34]:
# Check the data type
type(df['issue_d_date'][1])

pandas._libs.tslibs.timestamps.Timestamp

In [35]:
# Assume we are now in December 2017
# Calculate the number of months since the issue date
df['mths_since_issue_d'] = round(pd.to_numeric((pd.to_datetime('2017-12-01') - df['issue_d_date']) / np.timedelta64(1, 'M')))

In [36]:
# Shows some descriptive statisics for the values of a column
df['mths_since_issue_d'].describe()

count    466285.000000
mean         51.255187
std          14.340154
min          36.000000
25%          41.000000
50%          47.000000
75%          57.000000
max         126.000000
Name: mths_since_issue_d, dtype: float64

## Discrete Variables

In [37]:
# View unique values in the 'grade' column
df['grade'].unique()

array(['B', 'C', 'A', 'E', 'F', 'D', 'G'], dtype=object)

In [38]:
# Apply one-hot encoding to the 'grade' column
pd.get_dummies(df['grade'])

,A,B,C,D,E,F,G
0,0,1,0,0,0,0,0
1,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0
4,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...
466280,0,0,1,0,0,0,0
466281,0,0,0,1,0,0,0
466282,0,0,0,1,0,0,0
466283,1,0,0,0,0,0,0


In [39]:
# Select columns with categorical data types ('object' or 'category')
df.select_dtypes(include=['object', 'category']).columns

Index(['term', 'grade', 'sub_grade', 'emp_title', 'emp_length',
       'home_ownership', 'verification_status', 'issue_d', 'loan_status',
       'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code',
       'addr_state', 'earliest_cr_line', 'initial_list_status', 'last_pymnt_d',
       'next_pymnt_d', 'last_credit_pull_d', 'application_type'],
      dtype='object')

In [40]:
# Display unique values for each column to identify categorical variables
#for col in df.select_dtypes(include=['object', 'category']).columns:
#    unique_vals = df[col].unique()
#    print(f"{col}: {unique_vals}")

In [41]:
# Create one-hot encoded DataFrames for several categorical variables
df_dummies = [pd.get_dummies(df['grade'], prefix='grade', prefix_sep=':'),
              pd.get_dummies(df['sub_grade'], prefix='sub_grade', prefix_sep=':'),
              pd.get_dummies(df['home_ownership'], prefix='home_ownership', prefix_sep=':'),
              pd.get_dummies(df['verification_status'], prefix='verification_status', prefix_sep=':'),
              pd.get_dummies(df['loan_status'], prefix='loan_status', prefix_sep=':'),
              pd.get_dummies(df['purpose'], prefix='purpose', prefix_sep=':'),
              pd.get_dummies(df['addr_state'], prefix='addr_state', prefix_sep=':'),
              pd.get_dummies(df['initial_list_status'], prefix='initial_list_status', prefix_sep=':')
             ]

In [42]:
# Combine all the one-hot encoded DataFrames into a single DataFrame
df_dummies = pd.concat(df_dummies, axis = 1)

In [43]:
# Check the type of the df_dummies variable
type(df_dummies)

pandas.core.frame.DataFrame

In [44]:
# Combine the original DataFrame with the one-hot encoded DataFrame
df = pd.concat([df, df_dummies], axis = 1)

In [45]:
# Get the column names of the DataFrame
df.columns.values

array(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       'emp_title', 'emp_length', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan',
       'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d',
       'last_credit_pull_d', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'application_type',
       'annual_inc_joint', 'dti_joint', 'verification_status_joint

## Handling Missing Values

In [46]:
# Set the option to display all rows in the DataFrame output
pd.options.display.max_rows = None

# Display the count of missing values for each column
df.isnull().sum()

id                                                                      0
member_id                                                               0
loan_amnt                                                               0
funded_amnt                                                             0
funded_amnt_inv                                                         0
term                                                                    0
int_rate                                                                0
installment                                                             0
grade                                                                   0
sub_grade                                                               0
emp_title                                                           27588
emp_length                                                          21008
home_ownership                                                          0
annual_inc                            

In [47]:
# Set the option to display up to 100 rows in the DataFrame output
pd.options.display.max_rows = 100

In [48]:
# Fill missing values in the 'total_rev_hi_lim' column with the corresponding values from the 'funded_amnt' column
df['total_rev_hi_lim'].fillna(df['funded_amnt'], inplace=True)

In [49]:
# Check the number of missing values in the 'total_rev_hi_lim' column
df['total_rev_hi_lim'].isnull().sum()

0

In [50]:
# Replace the missing values with the mean of the non-missing values of the variable
df['annual_inc'].fillna(df['annual_inc'].mean(), inplace=True)

In [51]:
# Check the number of missing values
df['annual_inc'].isnull().sum()

0

In [52]:
# List of columns to replace missing values with zeros
columns_to_replace = ['mths_since_earliest_cr_line', 
                      'acc_now_delinq', 
                      'total_acc', 
                      'pub_rec', 
                      'open_acc', 
                      'inq_last_6mths', 
                      'delinq_2yrs', 
                      'emp_length_int']

In [53]:
# Replace missing values with zeros for the specified columns
df[columns_to_replace] = df[columns_to_replace].fillna(0)

In [54]:
# Check the number of missing values
df[columns_to_replace].isnull().sum()

mths_since_earliest_cr_line    0
acc_now_delinq                 0
total_acc                      0
pub_rec                        0
open_acc                       0
inq_last_6mths                 0
delinq_2yrs                    0
emp_length_int                 0
dtype: int64

# PD Model

## Data Preparation

In [55]:
# Dependent variable of PD model = indicator for default/non-default
df['loan_status'].unique()

array(['Fully Paid', 'Charged Off', 'Current', 'Default',
       'Late (31-120 days)', 'In Grace Period', 'Late (16-30 days)',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off'],
      dtype=object)

In [56]:
df['loan_status'].value_counts()

Current                                                224226
Fully Paid                                             184739
Charged Off                                             42475
Late (31-120 days)                                       6900
In Grace Period                                          3146
Does not meet the credit policy. Status:Fully Paid       1988
Late (16-30 days)                                        1218
Default                                                   832
Does not meet the credit policy. Status:Charged Off       761
Name: loan_status, dtype: int64

In [58]:
df['loan_status'].value_counts() / df['loan_status'].count()

Current                                                0.480878
Fully Paid                                             0.396193
Charged Off                                            0.091092
Late (31-120 days)                                     0.014798
In Grace Period                                        0.006747
Does not meet the credit policy. Status:Fully Paid     0.004263
Late (16-30 days)                                      0.002612
Default                                                0.001784
Does not meet the credit policy. Status:Charged Off    0.001632
Name: loan_status, dtype: float64

In [ ]:
df['good_bad'] = np.where(df['loan_status'].isin(['Charged Off', 
                                                  'Default',
                                                  'Does not meet the credit policy. Status:Charged Off',
                                                  'Late (31-120 days)'
                                                 ]))